In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from model_lora import load_lora, apply_lora, save_lora, LoRA

In [2]:
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(16, 16)  # square → LoRA will be added
        self.fc2 = nn.Linear(16, 8)   # NOT square → no LoRA here

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [3]:
model = SimpleModel().to("cuda")
apply_lora(model, rank=4)

In [4]:
model

SimpleModel(
  (fc1): Linear(
    in_features=16, out_features=16, bias=True
    (lora): LoRA(
      (A): Linear(in_features=16, out_features=4, bias=False)
      (B): Linear(in_features=4, out_features=16, bias=False)
    )
  )
  (fc2): Linear(in_features=16, out_features=8, bias=True)
)

In [5]:
print("\nLoRA attached? ", hasattr(model.fc1, "lora"))


LoRA attached?  True


In [6]:
print("LoRA attached to fc2? ", hasattr(model.fc2, "lora"))

LoRA attached to fc2?  False


In [7]:
# Create a random input
x = torch.randn(1, 16).to("cuda")

In [8]:
print("\n=== Forward pass (with LoRA) ===")
out1 = model(x)
print("Output before modifying LoRA:\n", out1)


=== Forward pass (with LoRA) ===
Output before modifying LoRA:
 tensor([[-0.0787,  0.1393, -0.1625, -0.4562,  0.4278, -0.0659, -0.1931, -0.2288]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [9]:
# Modify LoRA weights so changes will be visible after load
print("\n=== Modify LoRA weights ===")
with torch.no_grad():
    model.fc1.lora.A.weight.fill_(0.123)
    model.fc1.lora.B.weight.fill_(0.456)


=== Modify LoRA weights ===


In [10]:
out2 = model(x)
print("Output after modifying LoRA:\n", out2)

Output after modifying LoRA:
 tensor([[-0.2364,  0.4143, -0.2315, -0.3902,  0.5948,  0.0542, -0.1554,  0.0761]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [11]:
save_lora(model, "lora_weights.pt")

In [12]:
# Create a NEW model and apply LoRA again
print("\n=== Create new model and load LoRA ===")
model2 = SimpleModel().to("cuda")
apply_lora(model2, rank=4)


=== Create new model and load LoRA ===


In [13]:
# Before loading: different output
out3 = model2(x)
print("New model output before loading LoRA:\n", out3)

New model output before loading LoRA:
 tensor([[ 0.1949,  0.1155, -0.3426, -0.3066, -0.2650, -0.4405,  0.1356,  0.1195]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [14]:
# Load LoRA weights
load_lora(model2, "lora_weights.pt")

In [15]:
# After loading: must match modified output
out4 = model2(x)
print("\nNew model output after loading LoRA:\n", out4)


New model output after loading LoRA:
 tensor([[ 0.1410,  0.4229, -0.5414, -0.3217, -0.4193, -0.3787,  0.2722,  0.2433]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
